# Unit 9 Spark ML

## Contents
```
9.1. Introduction
  9.1.1. Machine Learning Basic Concepts
  9.1.2. Data Driven Decision Making
  9.1.3. Supervised vs Unsupervised Learning
  9.1.4. Data Science Life Cycle
  
9.2. Transformers

9.3. Estimators

9.4. Pipelines

9.5. Evaluators

9.6. Hyperparameter Tuning

9.7. Data Science Lifecycle in Spark
```

## Introduction

![AI vs ML vs DL](https://bigdata.cesga.es/img/spark_ml_ai_ml_dl.png)

From Wikipedia:
- Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by animals and humans. AI research has been defined as the field of study of intelligent agents, which refers to any system that perceives its environment and takes actions that maximize its chance of achieving its goals.
- Machine learning (ML) is a field of inquiry devoted to understanding and building methods that 'learn', that is, methods that leverage data to improve performance on some set of tasks. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications, such as in medicine, email filtering, speech recognition, and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks. A subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers, but not all machine learning is statistical learning. (for example Deep Learning is not statistical learning but it is part of ML)
- Deep learning (DL) is part of a broader family of machine learning methods based on artificial neural networks with representation learning.

Deep Learning is inspired in how brain works and has gained a lot of popularity in recent years, probing to be very successful in tasks like image classification, object recognition, speech recognition or language translation.

## Data Driven Decision Making

![Data Driven Decision Making](https://bigdata.cesga.es/img/spark_ml_data_driven_decision_making.png)

## Machine Learning Basic Concepts

- **Dataset**: the data that we have available about the problem we want to solve. It consists of a list of observations. It can be seen as table.

- **Observation**: each row of the dataset.

Each observation is divided in:

- **Features**: the attributes of the observation.
- **Label**: the output (result) of the observation (not always available, ie. unsupervised learning)

The dataset is divided in:

- **Training data**: the part of the observations that we use to train the model.  is a portion of the observations that train an ML algorithm to produce a model. A general practice is to split the collected data into three portions: training data, validation data, and test data. The test data portion is roughly about 70% or 80% of the original dataset.

- **Test data**: the part of the observations that we use to evaluate the performance of the model.

The usual split is 80% training, 20% testing.

- **Validation data**: a further split of the dataset that can be used for model tuning (a.k.a. hyperparameter tuning).

- **Model**: it refers to the algorithm that "learns" from the training data and is able to make predictions.
- **Prediction**: the label that the model predicts for a given set of features.


## Supervised vs Unsupervised Learning

There are two main approaches in machine learning: supervised learning and unsupervised learning.

The main difference between both approaches is the availability of labeled data. In supervised learning we use a dataset with labeled data, but in unsupervised learning we do not have labels in our observations.

Unsupervised learning is in general more difficult to do.

Supervised Learning:
- Classification: 
- Regression

Unsupervised Learning:
- Clustering
- Collaborative Filtering
- Dimensionality reduction (eg. PCA)

## Data Science Life Cycle

![Data Science Life Cycle](http://bigdata.cesga.es/img/spark_ml_datascience_lifecycle.png)

## Loading data

To load data we use the methods we already know:

In [2]:
reviews = spark.read.json('/tmp/reviews_Books_5_small.json')

To split data we use the `randomSplit` method:

In [3]:
trainingData, testData = reviews.randomSplit([0.8, 0.2])

## Overview: Transformers, Estimators and Evaluators

![Overview](http://bigdata.cesga.es/img/spark_ml_overview_transformer_estimator_evaluator.png)

## Transformers

A Transformer is an algorithm which can transform one DataFrame into another DataFrame.

Common Tranformers:
- Binarizer
- Bucketizer
- VectorAssembler
- Tokenizer
- StopWordsRemover
- HashingTF

**Binarizer**: A transformer to convert numerical features to binary (0/1) features

In [ ]:
from pyspark.ml.feature import Binarizer

binarizer = Binarizer(threshold=2.5, inputCol='overall', outputCol='label')

**Tokenizer**: A transformer that converts the input string to lowercase and then splits it by white spaces.

In [ ]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")

**StopWordsRemover**: A transformer that filters out stop words from input. Note: null values from input array are preserved unless adding null to stopWords explicitly.

In [ ]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")

**HashingTF**: A Transformer that converts a sequence of words into a fixed-length feature Vector. It maps a sequence of terms to their term frequencies using a hashing function.

In [ ]:
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(inputCol=remover.getOutputCol(), outputCol="features")

## Estimators

An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer.

Estimators have a `fit` method.

To use an estimator first we have to use its `fit` method on some input data:
```
model = estimator.fit(input_data)
```

After that we will have a Transformer that we can use to generate estimations:
```
estimations = model.transform(data)
```

There two type of Estimators
- The ones that are Machine Learning algorithms
- The ones that are a sort of complex Transformers that require a previous fit step: this are known as Data Transformation Estimators

For example the LogisticRegression Estimator is a classification algoritm based on the Logistic Regression algorithm, and the the MinMaxScaler is a data transformation estimator.

Machine Learning Algorithms:
- LogisticRegression
- DecisionTreeClassifier
- RandomForestClassifier
- LinearRegression
- RandomForestRegressor
- KMeans
- LDA
- BisectingKMeans
- ALS (recommendations a.k.a. collaborative filtering)

Data Transformation Algorithms:
- MixMaxScaler
- StringIndexer
- OneHotEncoderEstimator
- StandardScaler
- MaxAbsScaler
- IDF
- Word2Vec

**LogisticRegression**: A classification algorithm. Supports binomial logistic regression and multinomial logistic regression.

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.01)

model = lr.fit(trainingData)

In [30]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([Row(features=Vectors.dense(1.0, 0.0), label=0.0),
                            Row(features=Vectors.dense(2.0, 0.0), label=0.0),
                            Row(features=Vectors.dense(0.0, 1.0), label=1.0),
                            Row(features=Vectors.dense(0.0, 2.0), label=1.0)])
df.show(truncate=False)

+---------+-----+
|features |label|
+---------+-----+
|[1.0,0.0]|0.0  |
|[2.0,0.0]|0.0  |
|[0.0,1.0]|1.0  |
|[0.0,2.0]|1.0  |
+---------+-----+



In [31]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.01)

model = lr.fit(df)

model.transform(df).show()

+---------+-----+--------------------+--------------------+----------+
| features|label|       rawPrediction|         probability|prediction|
+---------+-----+--------------------+--------------------+----------+
|[1.0,0.0]|  0.0|[2.48197057737007...|[0.92286818542131...|       0.0|
|[2.0,0.0]|  0.0|[4.96394115474013...|[0.99306311334180...|       0.0|
|[0.0,1.0]|  1.0|[-2.4819705773700...|[0.07713181457868...|       1.0|
|[0.0,2.0]|  1.0|[-4.9639411547401...|[0.00693688665819...|       1.0|
+---------+-----+--------------------+--------------------+----------+



In [34]:
test = spark.createDataFrame([Row(label=1.0, weight=1.0, features=Vectors.dense(20.0, 0.0)),
                              Row(label=1.0, weight=1.0, features=Vectors.dense(0.0, 20.0))])
test.show()

+----------+-----+------+
|  features|label|weight|
+----------+-----+------+
|[20.0,0.0]|  1.0|   1.0|
|[0.0,20.0]|  1.0|   1.0|
+----------+-----+------+



In [35]:
model.transform(test).show()

+----------+-----+------+--------------------+--------------------+----------+
|  features|label|weight|       rawPrediction|         probability|prediction|
+----------+-----+------+--------------------+--------------------+----------+
|[20.0,0.0]|  1.0|   1.0|[49.6394115474013...|[1.0,2.7661611663...|       0.0|
|[0.0,20.0]|  1.0|   1.0|[-49.639411547401...|[2.76616116630809...|       1.0|
+----------+-----+------+--------------------+--------------------+----------+



**MinMaxScaler**: Rescale each feature individually to a common range [min, max] linearly using column summary statistics, which is also known as min-max normalization

Imagine that we have a dataset with the values of the temperature of a set of servers in a datacenter. For each server we have two temperatures: the inlet air temperature and the cpu temperature:

In [22]:
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([(Vectors.dense([24.0, 60.0]),),
                            (Vectors.dense([25.0, 62.0]),),
                            (Vectors.dense([25.5, 63.0]),),
                            (Vectors.dense([30.0, 64.0]),)], ["temperature"])
df.show()

+-----------+
|temperature|
+-----------+
|[24.0,60.0]|
|[25.0,62.0]|
|[25.5,63.0]|
|[30.0,64.0]|
+-----------+



In [23]:
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(inputCol='temperature', outputCol='temperature_scaled')
model = scaler.fit(df)

Once fit the MinMaxScaler was able to know the min and max of the data:

In [24]:
model.originalMin

DenseVector([24.0, 60.0])

In [25]:
model.originalMax

DenseVector([30.0, 64.0])

And now it can act as a Transformer:

In [27]:
model.transform(df).show(truncate=False)

+-----------+-------------------------+
|temperature|temperature_scaled       |
+-----------+-------------------------+
|[24.0,60.0]|[0.0,0.0]                |
|[25.0,62.0]|[0.16666666666666666,0.5]|
|[25.5,63.0]|[0.25,0.75]              |
|[30.0,64.0]|[1.0,1.0]                |
+-----------+-------------------------+



## Pipelines

A Pipeline is useful to chain multiple Transformers and Estimators together so we can create a workflow.

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[binarizer, tokenizer, remover, hashingTF, lr])

model = pipeline.fit(trainingData)

## Evaluators

Evaluators allow us to obtain model evaluation metrics that will help us to measure how well a fitted model performs on test data.

Since the evaluation metrics to calculate depend on the model we are trying to evaluate, we will have to select a evaluator that corresponds to the problme type we are solving.

**BinaryClassificationEvaluator**: evaluates the performance of binary classification models

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

predictions = pipeLineModel.transform(testData)

aur = evaluator.evaluate(predictions)

**MulticlassClassificationEvaluator**: evaluates the performance of multiclass classification models

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator()

predictions = pipeLineModel.transform(testData)

accuracy = evaluator.evaluate(predictions)

**RegressionEvaluator**: evaluates the performance of regression models as well as collaborative filtering models like ALS

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator()

predictions = pipeLineModel.transform(testData)

rmse = evaluator.evaluate(predictions)

## Hyperparameter Tuning

**CrossValidator**: K-fold cross validation performs model selection by splitting the dataset into a set of non-overlapping randomly partitioned folds which are used as separate training and test datasets e.g., with k=3 folds, K-fold cross validation will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. Each fold is used as the test set exactly once.

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
param_grid = ParamGridBuilder() \
            .addGrid(hashingTF.numFeatures, [10000, 100000]) \
            .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
            .addGrid(lr.maxIter, [10, 20]) \
            .build()
            
cv = (CrossValidator()
      .setEstimator(pipeline)
      .setEvaluator(evaluator)
      .setEstimatorParamMaps(param_grid)
      .setNumFolds(3))

cv_model = cv.fit(trainingData)

From the point of view of the Spark API a CrossValidator is just another Estimator:

![Overview](http://bigdata.cesga.es/img/spark_ml_crossvalidator.png)

## Data Science Life Cycle in Spark

![Data Science Life Cycle](http://bigdata.cesga.es/img/spark_ml_spark_lifecycle.png)

## Exercises

* Lab 1: Sentiment Analysis Amazon Books (short version)
* Lab 2: Titanic
* Lab 3: House prices
* Lab 4: Movielens

To help you during the labs you will find these references useful.

You can find the details of the different algorithms implemented in Spark in the following references:
- [Spark ML: Classification and regression](https://spark.apache.org/docs/latest/ml-classification-regression.html)
- [Spark ML: Clustering](https://spark.apache.org/docs/latest/ml-clustering.html)
- [Spark ML: Collaborative filtering](https://spark.apache.org/docs/latest/ml-collaborative-filtering.htmll)
- [Spark ML: Model selection and tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

And in case you have to deal with some legacy application implemented the old MLlib (RDD-based API) you will find useful this reference:
- [MLlib: RDD-based API Guide](https://spark.apache.org/docs/latest/mllib-guide.html)